# Projet ia 

In [116]:
import numpy as np
import matplotlib.pyplot as plt
from mido import MidiFile, MidiTrack, Message, MetaMessage
from scipy import linalg
from ipywidgets import *
from IPython.display import *

#Pour afficher les courbes dans le notebook
%matplotlib inline

## Import des notes du fichier midi désiré

In [117]:
noteon = []
#importation du fichier midi désiré
mid = MidiFile('midi-classique-flute/[Free-scores.com]_ali-choobdar-night-the-lotus-66972.midi')
for i, track in enumerate(mid.tracks):
    for msg in track:
        if msg.type == 'note_on':
            newNote = (msg.note,msg.time)
            #on ajoute chaque note dans le tableau noteon (hauteur + durée)
            noteon.append(newNote)
            print(noteon)

[(76, 49)]
[(76, 49), (65, 40)]
[(76, 49), (65, 40), (67, 494)]
[(76, 49), (65, 40), (67, 494), (69, 486)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70, 469)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70, 469), (76, 17)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70, 469), (76, 17), (74, 40)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70, 469), (76, 17), (74, 40), (67, 40)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70, 469), (76, 17), (74, 40), (67, 40), (69, 44)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70, 469), (76, 17), (74, 40), (67, 40), (69, 44), (70, 40)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70, 469), (76, 17), (74, 40), (67, 40), (69, 44), (70, 40), (74, 40)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70, 469), (76, 17), (74, 40), (67, 40), (69, 44), (70, 40), (74, 40), (81, 44)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70, 469), (76, 17), (74, 40), (67, 40), (69, 44), (70, 40), (74, 40), (81, 44), (62, 27)]
[(76, 49), (65, 40), (67, 494), (69, 486), (70,

# Utilisation du code de Word generator 

In [119]:
def set_seed(seed=None):
    """Making the seed (for random values) variable if None"""

    if seed is None:
        import time
        seed = int((time.time()*10**6) % 4294967295)
        print(seed)
    try:
        np.random.seed(seed)
        print("Seed used for random values:", seed)
    except:
        print("!!! WARNING !!!: Seed was not set correctly.")
    return seed

class Network(object):

    def __init__(self, trainLen=2000, testLen=2000, initLen=100) :
        self.initLen = initLen
        self.trainLen = trainLen
        self.testLen = testLen
        # données d'entrée :
        self.input_text = noteon #The one to keep
        self.resSize = 300
        self.a = 0.3
        self.spectral_radius = 1.25
        self.input_scaling = 1.
        self.reg =  1e-8
        self.mode = 'prediction'
        seed = None #42
        set_seed(seed)
        
nw = Network()

1868999957
Seed used for random values: 1868999957


Fonction permettant de définir les entrées et sorties de notre réseau de neurone 

In [120]:
def notes(nw) :
    nw.input_units, nw.output_units = dict(), dict()
    for i, item in enumerate(set(nw.input_text)) : nw.input_units[item] = i
    for i, item in enumerate(set(nw.input_text)) : nw.output_units[i] = item
    print("\nExisting notes in the music :", sorted(nw.input_units),
          "\nNumber of different notes :", len(nw.input_units), "\n")
    return(nw)

Code Word generator 

In [121]:
def convert_input(nw) :
    print("Converting input into ID numbers...")
    nw.data = np.array([nw.input_units[i] for i in nw.input_text])
    print(nw.data)
    nw.inSize = nw.outSize = len(nw.input_units)
    print("done.")
    return(nw)

In [122]:
def binary_data(nw) :
    print("Creating the input binary matrix...")#, end=" ")
    nw.data_b = np.zeros((len(nw.input_text), len(nw.input_units)))
    for i, item in enumerate(nw.data) :
        nw.data_b[i][item] = 1
    print(nw.data_b)
    print("done.\n") 
    return(nw)

## Initialisation du réseau (Word generator)

In [123]:
def initialization(nw) :
    print("\nInitializing the network matrices...")#, end=" ")
    set_seed()
    nw.Win = (np.random.rand(nw.resSize,1+nw.inSize)-0.5) * nw.input_scaling
    nw.W = np.random.rand(nw.resSize,nw.resSize)-0.5 
    nw.X = np.zeros((1+nw.inSize+nw.resSize,nw.trainLen-nw.initLen))
    nw.Ytarget = nw.data_b[nw.initLen+1:nw.trainLen+1].T
    nw.x = np.zeros((nw.resSize,1))
    print("done.")
    return(nw)

def compute_spectral_radius(nw):
    print('Computing spectral radius...')#,end=" ")
    rhoW = max(abs(linalg.eig(nw.W)[0]))
    print('done.')
    nw.W *= nw.spectral_radius / rhoW
    return(nw)

def train_network(nw) :
    print('Training the network...')#, end=" ")
    percent = 0.1
    for t in range(nw.trainLen):
        percent = progression(percent, t, nw.trainLen)
        nw.u = nw.data_b[t%len(nw.data)]
        nw.x = (1-nw.a)*nw.x + nw.a*np.tanh( np.dot(nw.Win, np.concatenate((np.array([1]),nw.u)).reshape(len(nw.input_units)+1,1) ) + np.dot( nw.W, nw.x ) )
        if t >= nw.initLen :
            nw.X[:,t-nw.initLen] = np.concatenate((np.array([1]),nw.u,nw.x[:,0])).reshape(len(nw.input_units)+nw.resSize+1,1)[:,0]      
    print('done.')
    return(nw)

def train_output(nw) :
    print('Training the output...')#, end=" ")
    nw.X_T = nw.X.T
    if nw.reg is not None:
        nw.Wout = np.dot(np.dot(nw.Ytarget,nw.X_T), linalg.inv(np.dot(nw.X,nw.X_T) + \
            nw.reg*np.eye(1+nw.inSize+nw.resSize) ) )
    else:
        nw.Wout = np.dot(nw.Ytarget, linalg.pinv(nw.X) )   
    print('done.')
    return(nw)

def test(nw) :
    print('Testing the network... (', nw.mode, ' mode)')#, sep="")#, end=" ")
    nw.Y = np.zeros((nw.outSize,nw.testLen))
    nw.u = nw.data_b[nw.trainLen%len(nw.data)]
    percent = 0.1
    for t in range(nw.testLen):
        percent = progression(percent, t, nw.trainLen)
        nw.x = (1-nw.a)*nw.x + nw.a*np.tanh( np.dot(nw.Win, np.concatenate((np.array([1]),nw.u)).reshape(len(nw.input_units)+1,1)\
                                                   ) + np.dot(nw.W,nw.x ) )
        nw.y = np.dot(nw.Wout, np.concatenate((np.array([1]),nw.u,nw.x[:,0])).reshape(len(nw.input_units)+nw.resSize+1,1)[:,0] )
        nw.Y[:,t] = nw.y
        if nw.mode == 'generative':
            # generative mode:
            nw.u = nw.y
        elif nw.mode == 'prediction':
            ## predictive mode:
            nw.u = np.zeros(len(nw.input_units))
            nw.u[nw.data[(nw.trainLen+t+1)%len(nw.data)]] = 1
        else:
            raise(Exception, "ERROR: 'mode' was not set correctly.")
    print('done.\n')
    return(nw)

def compute_error(nw) :
    print("Computing the error...")#, end=" ")
    errorLen = 50
    mse = sum( np.square( nw.data[(nw.trainLen+1)%len(nw.data):(nw.trainLen+errorLen+1)%len(nw.data)] - nw.Y[0,0:errorLen] ) ) / errorLen
    print('MSE = ' + str( mse ))
    return(nw)

In [124]:
def convert_output(nw) :
    nw.output_text = []
    print("Converting the output...")
    for i in range(len(nw.Y.T)) :
        nw.output_text.append(nw.output_units[nw.Y.T[i].argmax()])
    print("done.")
    return(nw.output_text)

In [64]:
def progression(percent, i, total) :
    if i == 0 :
        print("Progress :")
        percent = 0.1
    elif (i/total) > percent :
        print(round(percent*100))
        print("%")
        percent += 0.1
    if total-i == 1 :
        print("100%")
    return(percent)

In [65]:
def compute_network(nw) :
    nw = notes(nw)
    nw = convert_input(nw)
    nw = binary_data(nw)
    nw = initialization(nw)
    nw = compute_spectral_radius(nw)
    nw = train_network(nw)
    nw = train_output(nw)
    nw = test(nw) 
    #nw = compute_error(nw)
    nw = convert_output(nw)
    return(nw)

## Définition des paramètres du réseau 

In [66]:
select_mode = ToggleButtons(description='Mode:',
    options=['prediction', 'generative'])
var1 = FloatSlider(value=100, min=0, max=1000, step=1, description='resSize')
var2 = FloatSlider(value=50, min=0, max=2000, step=1, description='initLen')
var3 = FloatSlider(value=100, min=0, max=5000, step=1, description='trainLen')
var4 = FloatSlider(value=100, min=0, max=5000, step=1, description='testLen')
var5 = FloatSlider(value=1.25, min=0, max=10, step=0.05, description='spectral radius')
var6 = FloatSlider(value=0.3, min=0, max=1, step=0.01, description='leak rate')
valid = Button(description='Valider')

def record_values(_) :
    clear_output()
    nw.mode=select_mode.value
    nw.resSize=int(var1.value)
    nw.initLen=int(var2.value)
    nw.trainLen=int(var3.value)
    nw.testLen=int(var4.value)
    nw.spectral_radius=float(var5.value)
    nw.a=float(var6.value)
    print("InitLen:", nw.initLen, "TrainLen:", nw.trainLen, "TestLen:", nw.testLen) 
    print("ResSize:", nw.resSize, "Spectral Radius:", nw.spectral_radius, "Leak Rate:", nw.a)
    compute_network(nw)
    return(nw)

display(select_mode)
display(var1)
display(var2)
display(var3)
display(var4)
display(var5)
display(var6)
display(valid)

valid.on_click(record_values)

InitLen: 50 TrainLen: 100 TestLen: 100
ResSize: 100 Spectral Radius: 1.25 Leak Rate: 0.3

Existing notes in the music : [(60, 44), (60, 984), (62, 27), (62, 40), (62, 44), (62, 70), (62, 1046), (64, 40), (64, 44), (64, 984), (64, 1046), (65, 1), (65, 5), (65, 40), (65, 44), (65, 70), (65, 984), (65, 1046), (66, 40), (66, 44), (67, 1), (67, 5), (67, 40), (67, 44), (67, 70), (67, 494), (67, 1018), (67, 1046), (68, 40), (68, 984), (68, 1046), (69, 1), (69, 5), (69, 40), (69, 44), (69, 49), (69, 486), (70, 5), (70, 40), (70, 44), (70, 70), (70, 469), (70, 978), (70, 984), (72, 40), (72, 44), (72, 70), (73, 40), (74, 40), (74, 1046), (75, 40), (75, 41), (75, 44), (75, 984), (76, 17), (76, 40), (76, 44), (76, 49), (77, 40), (77, 44), (78, 40), (78, 44), (79, 40), (79, 44), (79, 70), (81, 40), (81, 44), (81, 984), (82, 39), (82, 40), (82, 44), (82, 1046), (84, 44), (84, 68)] 
Number of different notes : 74 

Converting input into ID numbers...
[13 11 73 61  8 68  0  9 52  4  0 15 42 45 11 10 

## Affichage des résultats en sortie de réseau

In [49]:
valid3 = Button(description='Show the output!')

def show_output(_) :
    print(nw.output_text)

display(valid3)
valid3.on_click(show_output)

Button(description='Show the output!', style=ButtonStyle())

[27, 27, 40, 27, 27, 27, 27, 27, 27, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 40, 40, 40, 40, 40, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 40, 27, 27, 27, 27, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44]
[(76, 40), (77, 44), (67, 40), (67, 40), (67, 40), (67, 40), (66, 40), (77, 44), (76, 40), (78, 40), (66, 40), (67, 40), (67, 40), (67, 40), (66, 40), (66, 40), (76, 40), (67, 40), (67, 40), (79, 44), (64, 40), (66, 40), (67, 40), (67, 40), (79, 44), (67, 40), (67, 40), (69, 5), (79, 44), (64, 40), (62, 44), (62, 44), (79, 44), (64, 40), (76, 40), (64, 44), (64, 44), (64, 44), (64, 44), (64, 44), (64, 44), (62, 1046), (76, 40), (64, 44), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (66, 40), (72, 40), (81, 44

## Création du fichier midi prédit par le réseau de neurones

In [114]:
def new_file():
    mid=MidiFile()
    track=MidiTrack()
    mid.tracks.append(track)
    track.append(MetaMessage('set_tempo', tempo = 1000000, time=0))
    # l'argument program indique l'instrument que l'on souhaite 
    track.append(Message('program_change', program=73, time=0))
    for n in nw.output_text:
        track.append(Message('note_on',note=n[0],velocity=30,time=n[1]))
        track.append(Message('note_off',note=n[0],velocity=30,time=n[1]))
    mid.save('monMorceau_pred_m_r.mid')

In [115]:
new_file()